In [2]:
#|default_exp core
#|export

import argparse, torch, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
#|export

def load_model(model_id: str):

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=False,
    )

    model.eval()

    return tokenizer, model


In [4]:
#|export

def run_model(tokenizer, model, prompt, max_new_tokens=128, temperature=0.7):

    device = model.device
    input = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **input,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
#|test

def response_generation_test():
    model = "unsloth/mistral-7b-instruct-v0.3"

    tokenizer, model = load_model(model)

    print(run_model(tokenizer, model, 'Hello, World!'))

response_generation_test()

In [1]:
#|export

def conversation_loop(model_id:str = "unsloth/mistral-7b-instruct-v0.3", window_size:int = 5):
    """Manages a conversation with a sliding window for history."""
    tokenizer, model = load_model(model_id)

    conversation_history = [
        {"role": "system", "content": "You are a friend having a conversation."}
    ]

    print("Perturbative LLM Cognition: An exploration of the metaphor of 'thinking'")
    print("Say hi to your friend, but careful he might be on something... (type: 'quit' to exit)\n")

    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "quit":
            print("Chatbot: Goodbye!")
            break

        # Append the new user message
        conversation_history.append({"role": "user", "content": user_input})

        # Keep only the most recent N user/assistant messages (plus the system prompt).
        if len(conversation_history) > window_size + 1:
            conversation_history = [conversation_history[0]] + conversation_history[-window_size:]

        # Apply the model's specific chat formatting template.
        full_prompt = tokenizer.apply_chat_template(
            conversation_history, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        response = run_model(tokenizer, model, full_prompt)
        
        # Extract the new assistant response from the full generated text.
        assistant_response = response.replace(full_prompt, "").strip()

        # Append the assistant's response to the history for the next turn.
        conversation_history.append({"role": "assistant", "content": assistant_response})
        
        print(f"Chatbot: {assistant_response}")



NameError: name 'load_model' is not defined